In [13]:
import pprint, json, math, os, sys
import fitz, pdfplumber, ocrmypdf
import pandas as pd
import numpy as np
from collections import defaultdict

# dir_path = "C:\\Users\\Kaustubh.keny\\Projects\\office-work\\mywork-repo"
# fund_path = "C:\\Users\\Kaustubh.keny\\Projects\\Mar 25"

dir_path = "C:\\Users\\rando\\OneDrive\\Documents\\mywork-repo"
fund_path = "E:\\PDFDrive\\Mar25"
sys.path.append(os.path.abspath(dir_path))

from app.utils import Helper
from app.parse_regex import *

dry_path = r'\data\output\DryRun.pdf'
fin_path = r'\data\input\financial_indices.xlsx'
mutual_fund = Helper.get_fund_paths(fund_path)

In [ ]:
def extract_clipped_data(input:str, pages:list, bboxes:list):
        
        document = fitz.open(input)
        final_list = []
    
        for pgn in pages:
            page = document[pgn]
            
            all_blocks = [] #store every data from bboxes
            
            for bbox in bboxes:
                blocks, seen_blocks = [], set()  #store unique blocks based on content and bbox
                
                page_blocks = page.get_text('dict', clip=bbox)['blocks']
                for block in page_blocks:
                    if block['type'] == 0 and 'lines' in block: #type 0 means text block
                        #hash_key
                        block_key = (tuple(block['bbox']), tuple(tuple(line['spans'][0]['text'] for line in block['lines'])))
                        if block_key not in seen_blocks:
                            seen_blocks.add(block_key)
                            blocks.append(block)

                sorted_blocks = sorted(blocks, key=lambda x: (x['bbox'][1], x['bbox'][0]))
                all_blocks.append(sorted_blocks)

            final_list.append({
                "pgn": pgn,
                "block": all_blocks #will be list[list,list,..]
            })

        document.close()
        return final_list
    
def extract_data_relative_line(path: str, line_x: float, side: str):
    doc = fitz.open(path)
    pages = doc.page_count

    final_list = []

    for pgn in range(pages):
        page = doc[pgn]

        blocks = page.get_text("dict")["blocks"]
        sorted_blocks = sorted(blocks, key=lambda x: (x["bbox"][1], x["bbox"][0]))
        extracted_blocks = []

        # Keep track of blocks to avoid duplicates
        added_blocks = set()

        for block in sorted_blocks:
            block_id = id(block)  # Unique identifier for the block

            for line in block.get("lines", []):
                for span in line.get("spans", []):
                    origin = span["origin"]
                    x0, _ = origin

                    # Check the side condition
                    if side == "left" and x0 < line_x and block_id not in added_blocks:
                        extracted_blocks.append(block)
                        added_blocks.add(block_id)  # Mark block as added
                    elif side == "right" and x0 > line_x and block_id not in added_blocks:
                        extracted_blocks.append(block)
                        added_blocks.add(block_id)  # Mark block as added

      
        final_list.append({
            "pgn": pgn,
            "blocks": extracted_blocks
        })

    doc.close()

    return final_list
  
def get_clipped_data(input:str, bboxes:list[set], *args):
    
        document = fitz.open(input)
        final_list = []
        if args:
            pages = list(args)
        else:
            pages = [i for i in document.page_count]
        
        for pgn in pages:
            page = document[pgn]

            blocks = []
            for bbox in bboxes:
                blocks.extend(page.get_text('dict', clip = bbox)['blocks']) #get all blocks
            
            filtered_blocks = [block for block in blocks if block['type']== 0 and 'lines' in block]
            # sorted_blocks = sorted(filtered_blocks, key= lambda x: (x['bbox'][1], x['bbox'][0]))
            
             # Extract text from sorted blocks
            extracted_text = []
            for block in filtered_blocks:
                block_text = []
                for line in block['lines']:
                    line_text = " ".join(span['text'] for span in line['spans'])
                    block_text.append(line_text)
                extracted_text.append("\n".join(block_text))
            
            final_list.append({
            "pgn": pgn,
            "block": filtered_blocks,
            "text": extracted_text
            })
            
            
        document.close()
        return final_list
    
def get_clipped_text(input:str, bboxes:list[set],*args):

    document = fitz.open(input)
    final_list = []
    
    if args:
        pages = list(args)
    else:
        pages = [i for i in document.page_count]
    
    for pgn in pages:
        page = document[pgn]
        blocks = []
        for bbox in bboxes:
            blocks = page.get_text('text', clip = bbox).split('\n') #get all blocks
        final_list.append({
        "pgn": pgn,
        "block": blocks
        })   
    document.close()
    return final_list

def get_proper_fund_names(path: str, pages: list):
    doc = fitz.open(path)
    title = {}

    for pgn in pages:
        page = doc[pgn]
        blocks = page.get_text("dict")['blocks']
        text_all = " ".join(
            span["text"].strip()
            for block in blocks[:4]
            for line in block.get("lines", [])
            for span in line.get("spans", [])
            if span["text"].strip()
        )

        text_all = re.sub(r'[^A-Za-z0-9\s]+', '', text_all).strip()
        matches = re.findall(r"((?:LIC\s*MF|BLNCED|LOW)\s+.*?(?:FUND|ETF|FTF|FOF|PLAN|SAVER))", text_all, re.IGNORECASE)

        title[pgn] = matches[0] if matches else ""

    return title

In [16]:
sample_path = mutual_fund["Shriram Mutual Fund"]
# sample_path = r"C:\Users\rando\OneDrive\Documents\Feb25\LIC Mutual Fund\25_28-Feb-25_FS_ocr.pdf"

In [17]:
lines = [
    ((200, 0), (200, 812)),# Vertical line
    ((360, 0), (360, 812)),
    # ((0, 145), (812, 145))
]
pages = [12, 14,16]
bboxes = [ [0, 100, 210, 812],[210, 100, 420, 812]] #[(0, 85, 180, 812),(180, 85, 360, 812),(0,100,270,812),(0,100,350,812)]
pages = [i for i in range(1,110)]
Helper.draw_lines_on_pdf(sample_path, lines, bboxes, pages, dir_path +dry_path)

Modified PDF saved to: C:\Users\rando\OneDrive\Documents\mywork-repo\data\output\DryRun.pdf


In [7]:
def extract_clipped_text_all_pages(pdf_path, clip_coords):
    results = {}
    doc = fitz.open(pdf_path)
    clip_rect = fitz.Rect(*clip_coords)
    try:
        for page_number, page in enumerate(doc):
            text = page.get_text("text", clip=clip_rect).strip()
            results[page_number] = text
    finally:
        doc.close()
    return results


In [ ]:

clip_coords = (484, 50,600, 100)  # (x0, y0, x1, y1)

clipped_texts = extract_clipped_text_all_pages(sample_path, clip_coords)

for page_num, text in clipped_texts.items():
    print(f"Page {page_num}:\n{text}\n{'-'*40}")


In [ ]:
def get_proper_fund_names(path: str):
    pattern = "(GROWW.*?(?:FUND|Fund|ETF\\s*(?:FOF)?))"
    title = {}   
    with fitz.open(path) as doc:
        for pgn, page in enumerate(doc):
            text = " ".join(page.get_text("text", clip=(0, 0, 470, 60)).split("\n"))
            text = re.sub("[^A-Za-z0-9\\s\\-\\(\\).,]+|\u2028", "", text).strip()
            # print(text)
            if matches := re.findall(pattern, text, re.DOTALL):
                title[pgn] = " ".join([_ for _ in matches[0].strip().split(" ") if _])
                print(pgn,matches[0])
    return title
title = get_proper_fund_names(sample_path)

In [ ]:
# r"(360 ONE.*)$" 0,0,520,50
# "(Aditya Birla.*(?:Plan\\*?\\#?\\'?|Sensex|Fund|Path|ETF|FOF\\*?|Scheme|EOF|Funds\\*?|Yojna)?)$" 0,0,470,25
# "(Bajaj.*(?:Fund|Path|ETF|FOF|EOF|Growth))$" 0,0,470,50
# "(Bank of India.*?(?:Plan|Funds?|ETF|FOF|FTF))"  0,55,280,105
# "(Baroda BNP.*?(?:Fund|Path|ETF|FTF|FOF|Index|Fund of Fund))" 0,0,220,120
# "CANARA.*?\\)" 0,0,400,55
# "((?:DSP|Bharat).*?(?:Fund\\s*(?:of Fund)?|FUND|ETF|FTF|FOF))" 0,0,500,40
# "(Edelweiss\s*.+?(?:Fund|Path|ETF|FOF|Path))" 0,0,150,100
# "((?:Franklin|Templeton).*?(?:Fund\\s*(?:of Funds)?|Plan))" 0,0,470,80
# "(GROWW.*?FUND)" 0,0,470,60
# "(HDFC.*?(?:FUND|Fund\\s*(?:of Funds?)?|ETF\\s*(?:Fund of Funds?)?))" 0,0,400,60
# "(HSBC.*?(?:FUND|Fund\\s*(?:of Funds?)?|ETF\\s*(?:Fund of Funds?)?))" 0,0,600,45
# "(Helios.*)" 0,0,600,40
# "((?:ICICI|BHARAT).*)" 0,0,490,30
# "(Invesco India.*?(?:Fund(?:of Funds?)?|ETF|FOF|Path))" 180,0,590,40
# "(ITI.*?(?:Fund(?:of Funds?)?|ETF|FOF|Path))" 0,30,300,100
# "(JM.*?(?:Fund(?:of Funds?)?|ETF|FOF|Path))" 0,0,400,50
# "(KOTAK.*?(?:FUND\\s*(?:OF FUNDS?\\s*|-\\s*\\w+)?|ETF|FOF|PATH|FTF))" 0,0,450,70
# "((?:LI?i?C|BSE|BANK|SMALL|HEALTH).*?(?:FUND|Path|ETF|FTF|EOF|FOF|PLAN|SAVER|FUND\s*OF\s*FUND))" 0,0,400,100
# "\\b(Mahindra.*?(?:Fund|ETF|EOF|FOF|FTF|Path|FO))\\b" 170,50,450,80
# r'MIRAE ASSET .*?\b(?:ETF|EOF|FOF|FTF|FUND|FUND OF FUND|INDEX FUND)\b' 0,0,560,140
# "(NJ.*?(?:FUND\\s*(?:OF FUNDS?\\s*|-\\s*\\w+)?|ETF|FOF|PATH|FTF))" 0,30,480,70
# "([A-Z0-9\\s\\-]+\\s*PGIM INDIA)" 0,0,300,80
# "(Samco.*?Fund)" 0,30,600,100
# SBI PASSIVE r"((?:SBI|i\s*_|S35).*$)" 0, 0, 400, 50 
# SBI NORMAL "(SBI.*?(?:Fund\\s*(?:of funds?|.*?Plan)?|ETF|FTF|FOF))" 160, 640, 550, 812
# r"(Union\s*[A-Za-z\s]+?(?:FUND|PATH|ETF|FOF))" 0,0,180,150 
# "(Sundaram.*?(?:Fund\\s*(?:of funds?|.*?Plan|Series)?|ETF|FTF|FOF))" 0,0,400,40
# "((?:Tata|Treasury|TATA).*?(?:Funds?(?:.*?Plan)?|ETF|FOF|EOF|Plan.*?(?:days)?))" 0,0,540,40
# "((?:TAURUS|Taurus).*?(?:FUND|Fund))" 0,0,480,40
# "(TRUSTMF.*?(?:FUND|Fund))" 0,0,340,45
# "(quant.*?(?:FUND|Fund))" 0,0,480,80
# "(UTI.*?(?:FUND(?:\\s*OF FUND|.+?DURATION)?|PLAN|ETF|FTF|FOF|Fund|ETF(?: FUND OF FUND)?)(?:\\s*\\(\\s*Erst.+?\\))?)" 0,0,460,35
# "(WhiteOak.*?(?:Fund))" 0,0,410,60
# "(Zerodha.*?(?:Fund|ETF(?:\\s*FoF)?|FoF))" 0,60,500,180
# "(Motilal.*?(?:Fund\\s*(?:o?O?f Funds?(?:.+?Aggressive|.+?Conservative)?)?|ETF(?:.+? of Funds?)?|FoF))" 0,0,600,60

# "(SHRIRAM.*?(?:FUNDS?|ETF|Fo?O?F|PLANS?))" 0,0,480,70

In [9]:
data = Helper.get_all_pdf_data(sample_path)

In [12]:
data[2]

{'pgn': 2,
 'blocks': [{'number': 5,
   'type': 0,
   'bbox': (74.03700256347656,
    109.62684631347656,
    355.2001037597656,
    147.6593475341797),
   'lines': [{'spans': [{'size': 27.65999984741211,
       'flags': 20,
       'font': 'NaviHeadline,Bold',
       'color': -19456,
       'ascender': 1.034999966621399,
       'descender': -0.3400000035762787,
       'text': 'Navi Flexi Cap Fund',
       'origin': (74.03700256347656, 138.25494384765625),
       'bbox': (74.03700256347656,
        109.62684631347656,
        355.2001037597656,
        147.6593475341797)}],
     'wmode': 0,
     'dir': (1.0, 0.0),
     'bbox': (74.03700256347656,
      109.62684631347656,
      355.2001037597656,
      147.6593475341797)}]},
  {'number': 6,
   'type': 0,
   'bbox': (74.03700256347656,
    148.2589569091797,
    356.99700927734375,
    178.01795959472656),
   'lines': [{'spans': [{'size': 12.0,
       'flags': 20,
       'font': 'NaviBody-DemiBold',
       'color': -19456,
       'ascend

In [ ]:
#canara
# def _update_manager_data(self,main_key:str,manager_data):
#     name = r"(?:Mr\.?|Mrs\.?|Ms\.?)\s+([A-Z][a-z]+\s[A-Z][a-z]+)"
#     exp = r"\b\d+\s*Years?\b"
#     since = r"\bSince\s*([0-9]+\s*-\s*[A-Za-z]+\.?\s*-\s*[0-9]+)\b"
#     nsample, msample, esample = [], [], []
#     nlength = 0
#     value = " ".join(manager_data.values())
#     nsample = re.findall(name, value, re.IGNORECASE)
#     esample = re.findall(exp, value, re.IGNORECASE)
#     msample = re.findall(since, value, re.IGNORECASE)
    
#     nlength = len(nsample)
#     msample += [""] * (nlength - len(msample))
#     esample += [""] * (nlength - len(esample))
    
#     final_list = [self._return_manager_data(since=m,name=n,exp=e)for n, m, e in zip(nsample, msample, esample)]
#     return {main_key:final_list}

# # hdfc
# def _extract_manager_data(self, main_key: str, data, pattern:str):
#     DATE_PATTERN = r"([A-Za-z]+\s*\d+),"
#     NAME_PATTERN = r"([A-Za-z]+\s[A-Za-z]+)"
#     EXP_PATTERN = r"over (\d+ years)"
#     YEAR_PATTERN = r"(\d{4})"

#     manager_data = " ".join(data)
#     manager_data = re.sub(r"[^A-Za-z0-9\s\-\(\).,]+", "", manager_data).strip()

#     experience_years = re.findall(EXP_PATTERN, manager_data, re.IGNORECASE)
#     dates = re.findall(DATE_PATTERN, manager_data, re.IGNORECASE)[:len(experience_years)]
#     years = re.findall(YEAR_PATTERN, manager_data, re.IGNORECASE)[:len(experience_years)]
#     names = re.findall(NAME_PATTERN, manager_data, re.IGNORECASE)[:len(experience_years)]
    
#     managing_since = [f"{date}, {year}" for date, year in zip(dates, years)]
#     experience_list = [f"{exp} years" for exp in experience_years]
#     final_list = [
#         self._return_manager_data(name=name,since=since,exp=exp)
#         for since, exp, name in zip(managing_since, experience_list, names)
#     ]
    
#     return {main_key:final_list}


# #NJMF
# def _update_manager_data(self,main_key:str,manager_data):
#     exp ="\\d{1,2} years"
#     name = "(?:Mr\\.?|Mrs\\.?|Ms\\.?)\\s*([A-Za-z]+\\s*[A-Za-z]+)"
#     since = "(?:since|from)\\s*([A-Za-z]+\\s*\\d{1,2},\\s*\\d{3,4}|inception)"
    
#     manager_data = " ".join(manager_data) if isinstance(manager_data,list) else manager_data
#     manager_data =re.sub(self.REGEX["escape"], "", manager_data).strip()
#     nsample = re.findall(name, manager_data, re.IGNORECASE)
#     esample = re.findall(exp, manager_data, re.IGNORECASE)
#     msample = re.findall(since, manager_data, re.IGNORECASE)
#     final_list = [self._return_manager_data(since=m,name=n,exp=e)for n, m, e in zip(nsample, msample, esample)]
#     return {main_key:final_list}

# #sbi
# def _extract_manager_data(self, main_key: str, data, pattern: str):
#     name = "(?:Mr\\.?|Mrs\\.?|Ms\\.?)\\s*([A-Za-z.]+\\s*[A-Za-z]+)"
#     since = "((?:\\(w.e.f\\.?)?[A-Za-z]+\\s*\\d{4}\\s*(?:\\()?)"
#     exp = "([0-9]+ years)"

#     final_list = []
#     manager_data = " ".join(data) if isinstance(data,list) else data
#     manager_data =re.sub(self.REGEX["escape"], "", manager_data).strip()
#     n = re.findall(name,manager_data, re.IGNORECASE)
#     s = re.findall(since,manager_data, re.IGNORECASE)
#     e = re.findall(exp,manager_data, re.IGNORECASE)
    
#     adjust = lambda target, lst: target[:len(lst)] + ([target[-1]] * abs(len(target) - len(lst)) if lst else [""])
#     n,s = adjust(n,e),adjust(s,e)
#     for name,since,exp in zip(n,s,e):
#         final_list.append(self._return_manager_data(name=name,since=since,exp=exp))
#     return {main_key: final_list}

# #sbI PASSIVE
# def _update_manager_data(self,main_key:str,data):
#     final_list = []
#     manager_data = " ".join(data) if isinstance(data, list) else data
#     manager_data = re.sub(self.REGEX['escape'], "", manager_data).strip()
#     pattern_info = self.REGEX["manager"]
#     regex_pattern = pattern_info['pattern']
#     field_names = pattern_info['fields']
#     if matches := re.findall(regex_pattern, manager_data, re.IGNORECASE):
#         for match in matches:
#             if isinstance(match, str):
#                 match = (match,)
#             record = {field_names[i]: match[i] if i < len(match) else "" for i in range(len(field_names))} #kwargs
#             final_list.append(self._return_manager_data(**record))

#     return {main_key: final_list}
    
# # ADITYA
# def _update_manager_data(self,main_key:str,manager_data):
#     nsample, msample, esample = [], [], []
#     nlength = 0
#     for key, value in manager_data.items():
#         if re.search(r"\bfund_manager\b", key, re.IGNORECASE):
#             nsample = re.findall(self.REGEX["manager"]["name"], value, re.IGNORECASE)
#             nlength = len(nsample)

#         elif re.search(r"^managing", key, re.IGNORECASE):
#             msample = re.findall(self.REGEX["manager"]["since"], value, re.IGNORECASE)
            
#         elif re.search(r"^experience", key, re.IGNORECASE):
#             esample = re.findall(self.REGEX["manager"]["exp"], value, re.IGNORECASE)
#     nlength = len(nsample)
#     msample += [""] * (nlength - len(msample))
#     esample += [""] * (nlength - len(esample))

#     final_list = [
#         self._return_manager_data(since=m,name=n,exp=e)
#         for n, m, e in zip(nsample, msample, esample)
#     ]

#     return {main_key:final_list}

In [ ]:
import camelot
def get_something(path: str):
    pattern = r"^(REDEMPTION PROCEEDS|FEATURES|ASSET ALLOCATION|FUND MANAGER|SCHEME|Sr\. No\.)$"

    with fitz.open(path) as doc:
        exists = defaultdict(int)
        for pgn, page in enumerate(doc):
            page_text = [t.strip() for t in page.get_text().split("\n")]
            for text in page_text:
                if re.match(pattern,text):
                    exists[pgn]+=1
                    
    
    return [str(pgn+1) for pgn, count in exists.items() if count > 4] #camelot starts pages from 1

pages = get_something(sample_path)

imp_pages = ",".join(pages)                

In [ ]:
tables = camelot.read_pdf(sample_path,pages=imp_pages, flavor="stream", table_areas= ["30,50,612,812"],column_tol = 4, split_text = True) #["30,50,612,812"]   

In [ ]:
import camelot
import pandas as pd
import numpy as np

def clean_column_name(x):
    cleaned = re.sub(r"[^A-Za-z0-9\s]", "", str(x), flags=re.IGNORECASE) #Corrected re.sub.
    cleaned = "_".join(cleaned.split())
    return cleaned

with pd.ExcelWriter("cleaned_tables.xlsx", engine="xlsxwriter") as writer:
    for i, table in enumerate(tables):
        df = table.df
        df = df.map(lambda x: " ".join(str(x).split("\n")).strip())
        df.replace("", np.nan, inplace=True)
        df.iloc[2:, 0] = df.iloc[2:, 0].ffill()
        df = df.iloc[1:, :]
        df.columns = df.iloc[0, :].apply(clean_column_name)
        # print(df.columns)
        df = df.iloc[1:, :]
        sheet_name = f"Table_{i+1}"

        # workbook = writer.book
        # worksheet = writer.sheets[sheet_name]

        # row = 1
        # last_value = None
        # start_row = None

        # for j, value in enumerate(df.iloc[:, 0], start=1):
        #     if pd.notna(value):  # New group found
        #         if last_value is not None and start_row is not None:
        #             worksheet.merge_range(start_row, 0, row - 1, 0, last_value)  # Merge previous block
        #         last_value = value
        #         start_row = row
        #     row += 1

        # # Merge last group
        # if last_value is not None and start_row is not None:
        #     worksheet.merge_range(start_row, 0, row - 1, 0, last_value)
    
        

print("Cleaned tables saved with merged spanning rows!")


In [ ]:
# fund_dict_cleaned = {
#     'LIC MF LARGE CAP FUND': 'LARGE CAP',
#     'LIC MF LARGE & MID CAP FUND': 'LARGE& MID CAP',
#     'LIC MF MULTICAP FUND': 'MULTICAP',
#     'LIC MF MID CAP FUND': "V't CAP",
#     'LIC MF SMALLCAP FUND': 'SMALLCAP',
#     'LIC MF DIVIDEND YIELD FUND': 'DIV_DEND YIELD',
#     'LIC MF VALUE FUND': 'VALUE',
#     'LIC MF FOCUSED FUND': 'FOCUSED',
#     'LIC MF INFRASTRUCTURE FUND': 'INFRASTRUCTURE',
#     'LIC MF MANUFACTURING FUND': 'Poin MANGFACTURING',
#     'LIC MF BANKING & FINANCIAL SERVICES FUND': 'BANKING & FINANC-AL SERVICES',
#     'LIC MF HEALTHCARE FUND': 'HEALTHCARE',
#     'LIC MF ELSS TAX SAVER': 'EL_SS TAX SAVER',
#     'LIC MF AGGRESSIVE HYBID FUND': 'AGGRESSIVE HYBRiD',
#     'LIC MF BALANCED ADVANTAGE FUND': 'B’L*NCED ADVANTAGE',
#     'LIC MF EQUITY SAVINGS FUND': 'EQUITY SAVINGS',
#     'LIC MF CONSERVATIVE HYBRID FUND': 'CONSERWATIVE nip?!)',
#     'LIC MF ARBITRAGE FUND': 'ARBITRAGE',
#     'LIC MF OVERNIGHT FUND': 'OVERNIGHT',
#     'LIC MF LIQUID FUND': 'LIQUID',
#     'LIC MF ULTRA SHORT DURATION FUND': 'ULTRA SHORT DURATION',
#     'LIC MF LOW DURATION FUND': "LOW’ DURATION",
#     'LIC MF MEDIUM TO LONG DURATION FUND': 'MEDIUM-F2°LONG DURATION',
#     'LIC MF BANKING & PSU FUND': 'BANK&ONG & PSU',
#     'LIC MF SHORT DURATION FUND': '_. SHORT DURATION',
#     'LIC MF GILT FUND': 'MGI',
#     'LIC MF CHILDRENS FUND': 'l HILDRENS',
#     'LIC MF BSE SENSEX ETF': 'BSE SENSEX ETF',
#     'LIC MF NIFTY 50 ETF': 'NIFTY 50ETF',
#     'LIC MF NIFTY 100 ETF': 'NIFTY 100 ETF',
#     'LIC MF NIFTY MIDCAP 100 ETF': 'NIFTY MIDCAP 100 ETF',
#     'LIC MF NIFTY 8-13 YR G-SEC ETF': 'NIFTY 8-13 YR G-SECETF',
#     'LIC MF BSE SENSEX INDEX FUND': 'BSE SENSEX INDEX',
#     'LIC MF NIFTY 50 INDEX FUND': 'NIFTY 50 INDEX',
#     'LIC MF NIFTY NEXT 50 INDEX FUND': 'NIFTY NEXT 50 INDEX',
#     'LIC MF GOLD EXCHANGE TRADED FUND': 'G2LD EXCHANGE TRADED',
#     'LIC MF GOLD ETF FUND OF FUND': 'GSLD ETF',
# }


In [ ]:
fund_dict_cleaned = {
    'LIC MF LARGE CAP FUND': 'LARGE CAP',
    'LIC MF LARGE & MID CAP FUND': 'LARGE.*?MID CAP',
    'LIC MF MULTICAP FUND': 'MULTICAP',
    'LIC MF MID CAP FUND': "V't CAP",
    'LIC MF SMALLCAP FUND': 'SMALLCAP',
    'LIC MF DIVIDEND YIELD FUND': 'DI.*?END YIELD',
    'LIC MF VALUE FUND': 'VALUE FUND',
    'LIC MF FOCUSED FUND': 'FOCUSED FUND',
    'LIC MF INFRASTRUCTURE FUND': 'INFRASTRUCTURE',
    'LIC MF MANUFACTURING FUND': 'MANGFACTURING',
    'LIC MF BANKING & FINANCIAL SERVICES FUND': 'BANKING.*?FINAN.*?SERVICES',
    'LIC MF HEALTHCARE FUND': 'HEALTHCARE',
    'LIC MF ELSS TAX SAVER': 'EL.*?TAX SAVER',
    'LIC MF AGGRESSIVE HYBID FUND': 'AGGRESSIVE HYBRiD',
    'LIC MF BALANCED ADVANTAGE FUND': 'B.*?NCED ADVANTAGE',
    'LIC MF EQUITY SAVINGS FUND': 'EQUITY SAVINGS',
    'LIC MF CONSERVATIVE HYBRID FUND': 'CONSERWATIVE.*nip?!)',
    'LIC MF ARBITRAGE FUND': 'ARBITRAGE FUND',
    'LIC MF OVERNIGHT FUND': 'OVERNIGHT',
    '':'LIC MF MULTI.*?CATION FUND',
    'LIC MF LIQUID FUND': 'LIQUID FUND',
    'LIC MF ULTRA SHORT DURATION FUND': 'ULTRA SHORT DURATION',
    'LIC MF LOW DURATION FUND': "LO.*?DURATION",
    'LIC MF MEDIUM TO LONG DURATION FUND': 'MEDIUM.*?LONG DURATION',
    'LIC MF BANKING & PSU FUND': 'BANK.*?PSU',
    'LIC MF SHORT DURATION FUND': 'SHORT DURATION',
    'LIC MF GILT FUND': 'MGI',
    'LIC MF CHILDRENS FUND': '.*?HILDRE.*?FUND',
    'LIC MF BSE SENSEX ETF': 'BSE SENSEX ETF',
    'LIC MF NIFTY 50 ETF': 'NIFTY\\s*50\\s*ETF',
    'LIC MF NIFTY 100 ETF': 'NIFTY 100 ETF',
    'LIC MF NIFTY MIDCAP 100 ETF': 'NIFTY MIDCAP 100 ETF',
    'LIC MF NIFTY 8-13 YR G-SEC ETF': 'NIFTY 8-13 YR G-SEC\\s*ETF',
    'LIC MF BSE SENSEX INDEX FUND': 'BSE SENSEX INDEX',
    'LIC MF NIFTY 50 INDEX FUND': 'NIFTY 50 INDEX',
    'LIC MF NIFTY NEXT 50 INDEX FUND': 'NIFTY NEXT 50 INDEX',
    'LIC MF GOLD EXCHANGE TRADED FUND': 'G2LD\\s*EXCHANGE\\s*TRADED',
    'LIC MF GOLD ETF FUND OF FUND': 'GSLD\s*ETF',
}


NIPPON DATA

In [ ]:
def via_block(path:str):
    pattern = r"FUNDS AT A GLANCE"
    amc_pattern = "^(Nippon India|CPSE).*(?=Plan|Next 50|Sensex|Fund|Path|ETF|FOF|EOF|Funds|$)"
    imp_pages = []
    with fitz.open(path) as doc:
        for pgn, page in enumerate(doc):
                page_blocks = page.get_text("dict")["blocks"]
                sorted_blocks = sorted(page_blocks, key=lambda x: (x["bbox"][1], x["bbox"][0]))
                for block_count, block in enumerate(sorted_blocks[:10]):
                    if "lines" not in block:
                        continue
                    for line in block["lines"]:
                        for span in line["spans"]:
                            text = span["text"].strip()
                            if re.match(pattern,text):
                                imp_pages.append(pgn)
                                
        amc_fund = defaultdict(list)
    
        for pgn in imp_pages:
            page = doc[pgn]
            page_blocks = page.get_text("dict")["blocks"]
            sorted_blocks = sorted(page_blocks, key=lambda x: (x["bbox"][1], x["bbox"][0]))
            for block_count, block in enumerate(sorted_blocks):
                if "lines" not in block:
                    continue
                for line in block["lines"]:
                    for span in line["spans"]:
                        text = span["text"].strip()
                        color = span['color']
                        if re.match(amc_pattern,text)and color == -1:
                            # matches = re.findall(amc_pattern,text)
                            amc_fund[pgn].append(text)
                            
    return imp_pages, dict(amc_fund)          

In [ ]:
pages,amc = via_block(sample_path)
pages = list(map(str,[x+1 for x in pages]))

In [ ]:
final_scheme = defaultdict(list)
for key, value in amc.items():
    # print(key)
    set1 = ['Scheme Name']+value[:4]
    set2 = ['Scheme Name']+value[4:]
    final_scheme[key+1].append(set1)
    final_scheme[key+1].append(set2)

final_scheme = dict(final_scheme)

In [ ]:
imp_pages = ",".join(pages)
tables = camelot.read_pdf(sample_path,pages=imp_pages, flavor="lattice", line_scale = 40)  #table_areas = ["0,0,580,690"]            

In [ ]:
import pandas as pd
import numpy as np

with pd.ExcelWriter("merged_tables.xlsx", engine="openpyxl") as writer:
    count = 0  # Toggle between 0 and 1
    
    for i, table in enumerate(tables):
        df = table.df
        if df.shape[1] < 3:
            continue

    
        df = df.map(lambda x: " ".join(x.split("\n")).strip())
        df = df.map(lambda x: np.nan if not x.strip() else x)
        df.set_index(df.columns[0], inplace=True)

        for check in ["Scheme Name", "Market Capitalization"]:
            if check in df.index:
                df.drop(check, inplace=True)
                
        df_cleaned = df[~df.index.isna()]
        df_cleaned = df_cleaned[df_cleaned.index != ""]
        df_cleaned = df_cleaned.reset_index()
        df_fill = df_cleaned.ffill(axis=1)


        sch_vals = final_scheme[table.page][count]
        count = 1 - count  # Toggle between 0 and 1

        if len(sch_vals) == 5:
            df_fill.loc[-1] = sch_vals 
            df_fill = df_fill.sort_index().reset_index(drop=True)

        # Write to a new sheet
        df_fill.to_excel(writer, sheet_name=f"Table_{i+1}", index=False)

print("All tables saved in separate sheets in 'merged_tables.xlsx' 🚀")


In [ ]:
hello =  {
    "number": 0,
    "type": 0,
    "bbox": (0,0,0,0), #406.72119140625, 439.4930419921875, 565.697265625, 484.5830383300781
    "lines": [
        {
            "spans": [
                {
                    "size": 30.0,
                    "flags": 20,
                    "font": "Montserrat-Regular", #set this
                    "color": -1, #set this
                    "ascender": 1.0429999828338623,
                    "descender": -0.2619999945163727,
                    "text": "DUMMYDUMMYDUMMYDUMMY",
                    "origin": (406.72119140625, 458.26702880859375),
                    "bbox": (0,0,0,0), #406.72119140625,439.4930419921875,565.697265625,462.9830322265625,
                }
            ],
            "wmode": 0,
            "dir": (1.0, 0.0),
            "bbox": (0,0,0,0), #406.72119140625,439.4930419921875,565.697265625,462.9830322265625,
        },
        
    ],
},  